In [1]:
import pickle
import glob
import os
import pandas as pd

In [23]:
os.chdir("pickles")

FileNotFoundError: [Errno 2] No such file or directory: 'pickles'

In [3]:
pkl_files = glob.glob("*.pkl")

In [4]:
len(pkl_files)

100

In [5]:
map_data = {}
for pkl_file in pkl_files:
    # lat_lng = (float(i) for i in pkl_file[:-4].split("_"))
    with open(pkl_file, "rb") as input:
        map_data[pkl_file] = pickle.load(input)

In [6]:
def get_admin_level_short_names(result, lvl):
    if len(result) == 1:
        return [i["short_name"] for i in result[0]["address_components"] if "administrative_area_level_{}".format(lvl) in i["types"]]
    else:
        return []

In [7]:
map_df = pd.DataFrame([[
        pkl_file,
        [float(i) for i in pkl_file[:-4].split("_")],
        type(map_data[pkl_file]),
        len(map_data[pkl_file]),
        get_admin_level_short_names(map_data[pkl_file], "1"),
        get_admin_level_short_names(map_data[pkl_file], "2")
    ] for pkl_file in pkl_files],
    columns = ["filename", "latlng", "result_type", "result_length", "admin_lvl_1_short_names", "admin_lvl_2_short_names"]
)

for lvl in [1, 2]:
    map_df["admin_lvl_{}_num".format(lvl)] = map_df["admin_lvl_{}_short_names".format(lvl)].map(len)

In [8]:
map_df.head()

,filename,latlng,result_type,result_length,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_1_num,admin_lvl_2_num
0,44.56777777777778_-75.40333333333334.pkl,"[44.56777777777778, -75.40333333333334]",<class 'list'>,1,[NY],[St Lawrence County],1,1
1,42.51888888888889_-71.83.pkl,"[42.51888888888889, -71.83]",<class 'list'>,1,[MA],[Worcester County],1,1
2,42.51888888888889_-75.40333333333334.pkl,"[42.51888888888889, -75.40333333333334]",<class 'list'>,1,[NY],[Chenango County],1,1
3,41.49444444444444_-74.51.pkl,"[41.49444444444444, -74.51]",<class 'list'>,1,[NY],[Orange County],1,1
4,43.54333333333333_-74.51.pkl,"[43.54333333333333, -74.51]",<class 'list'>,1,[NY],[Hamilton County],1,1


Make sure all result types are list

In [9]:
map_df.groupby("result_type").count()

,filename,latlng,result_length,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_1_num,admin_lvl_2_num
result_type,,,,,,,
<class 'list'>,100,100,100,100,100,100,100


Check lengths of results

In [10]:
map_df.groupby("result_length").count()

,filename,latlng,result_type,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_1_num,admin_lvl_2_num
result_length,,,,,,,
0,13,13,13,13,13,13,13
1,87,87,87,87,87,87,87


Check lengths of admin short names

In [11]:
map_df.groupby("admin_lvl_1_num").count()

,filename,latlng,result_type,result_length,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_2_num
admin_lvl_1_num,,,,,,,
0,13,13,13,13,13,13,13
1,87,87,87,87,87,87,87


In [12]:
map_df.groupby("admin_lvl_2_num").count()

,filename,latlng,result_type,result_length,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_1_num
admin_lvl_2_num,,,,,,,
0,13,13,13,13,13,13,13
1,87,87,87,87,87,87,87


If all counts look good, add lat, long, admin level names

In [13]:
map_df["lat"] = map_df.latlng.map(lambda x: x[0])

In [14]:
map_df["lng"] = map_df.latlng.map(lambda x: x[1])

In [15]:
map_df["state"] = map_df.admin_lvl_1_short_names.map(lambda x: x[0] if len(x) > 0 else "")

In [16]:
map_df["county"] = map_df.admin_lvl_2_short_names.map(lambda x: x[0] if len(x) > 0 else "")

In [18]:
map_df.groupby("state").count()

,filename,latlng,result_type,result_length,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_1_num,admin_lvl_2_num,lat,lng,county
state,,,,,,,,,,,
,13,13,13,13,13,13,13,13,13,13,13
CT,4,4,4,4,4,4,4,4,4,4,4
MA,2,2,2,2,2,2,2,2,2,2,2
NH,3,3,3,3,3,3,3,3,3,3,3
NJ,2,2,2,2,2,2,2,2,2,2,2
NY,31,31,31,31,31,31,31,31,31,31,31
ON,18,18,18,18,18,18,18,18,18,18,18
PA,19,19,19,19,19,19,19,19,19,19,19
QC,3,3,3,3,3,3,3,3,3,3,3


In [20]:
map_df.groupby("county").count().head()

,filename,latlng,result_type,result_length,admin_lvl_1_short_names,admin_lvl_2_short_names,admin_lvl_1_num,admin_lvl_2_num,lat,lng,state
county,,,,,,,,,,,
,13,13,13,13,13,13,13,13,13,13,13
Allegany County,2,2,2,2,2,2,2,2,2,2,2
Allegheny County,1,1,1,1,1,1,1,1,1,1,1
Berks County,1,1,1,1,1,1,1,1,1,1,1
Brome-Missisquoi Regional County Municipality,1,1,1,1,1,1,1,1,1,1,1


In [25]:
with open("./../ny_map_data.json", "w") as output:
    output.write(map_df[["lat", "lng", "state", "county"]].to_json(orient='records'))